# Case study - Toronto neighborhood analysis
## Part 1 - Solution start
###1. Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto

### Importing libraries

In [23]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering library
from geopy.geocoders import Nominatim

# import k-means from clustering stage
from sklearn.cluster import KMeans

print("Libraries imported.")

Libraries imported.


### Wikipedia scraping

In [24]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [25]:
# parse data from the html into a beautifulsoup object
#soup = BeautifulSoup(data, 'html.parser')
soup = BeautifulSoup(data, 'lxml')

In [26]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [27]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
toronto_df=pd.DataFrame(table_contents)
toronto_df['Borough']=toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [28]:
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [29]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [30]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [31]:

# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [32]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [33]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

# Part 2 solution start
### 1. Get the Geospatial data and add it to the dataframe

In [34]:
# load the coordinates from the csv file on Coursera
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [37]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


# Part 3 Solution start
### Explore and cluster the neighborhoods in Toronto (replicate the same analysis we did to New York City data).

In [38]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top before filtering for toronto

In [39]:
# create map of New York using latitude and longitude values before filtering for toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [40]:
CLIENT_ID = 'Y5DV33TWWVM1QZ15EDAU4EFRXXWXTSG5ASB1GPKDVEHYQRIX' # your Foursquare ID
CLIENT_SECRET = 'OJSREK0AOUPGSQXPUABGQMPFSGPB5QKU44T1AWKXEWWVEB44' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

In [41]:
toronto_df_new.loc[0, 'Neighborhood']

'Malvern, Rouge'

In [42]:
# filter borough names that contain the word Toronto
borough_names_unique = list(toronto_df_new.Borough.unique())

borough_with_toronto_name = []

for x in borough_names_unique:
    if "toronto" in x.lower():
        borough_with_toronto_name.append(x)
        
borough_with_toronto_name

['East Toronto',
 'East York/East Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'Downtown Toronto Stn A',
 'West Toronto',
 'East Toronto Business']

In [43]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto_name)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,M4M,East Toronto,Studio District,43.659526,-79.340923


### Create a map of Toronto with neighborhoods superimposed on top after filtering for toronto

In [44]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Test the API call to foursquare

In [45]:
radius = 500
LIMIT = 100
lat = 43.628947
lng = -79.394420
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)

print(url)

results = requests.get(url).json()

results

https://api.foursquare.com/v2/venues/explore?&client_id=Y5DV33TWWVM1QZ15EDAU4EFRXXWXTSG5ASB1GPKDVEHYQRIX&client_secret=OJSREK0AOUPGSQXPUABGQMPFSGPB5QKU44T1AWKXEWWVEB44&v=20180604&ll=43.628947,-79.39442&radius=500&limit=100


{'meta': {'code': 200, 'requestId': '60dab6d952ef4f7b8ff55d1f'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 43.6334470045, 'lng': -79.38821461519161},
   'sw': {'lat': 43.624446995499994, 'lng': -79.40062538480838}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad8df54f964a520881521e3',
       'name': 'Billy Bishop Toronto City Airport (YTZ) (Billy Bishop Toronto City Airport)',
       'location': {'address': 'Toronto Island',
        'lat': 43.63168259661481,
        'lng': -79.3960334124689,
        'distance': 331,
        'postalCode': 'M5J 1B7',
        'cc': 'CA',
        'city': 'Toronto',
        'state': 'ON',
        'country':

### Getting Venue data

In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [47]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-47-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Billy Bishop Toronto City Airport (YTZ) (Billy...,Airport,43.631683,-79.396033
1,Toronto Harbour,Harbor / Marina,43.633045,-79.396484
2,Porter Lounge,Airport Lounge,43.630680,-79.395756
3,Billy Bishop Café,Airport Food Court,43.631132,-79.396139
4,Air Canada Check-In Counter,Airport Terminal,43.631226,-79.395987


In [48]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


### Getting data from foursquare for all Latitude and longitude for Toronto and then appending the postal code, borough and Neighborhood data

In [49]:
radius=500
LIMIT = 100

venues_list=[]
#for lat, long, post in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode']):
for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
         
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']


        # return only relevant information for each nearby venue
        for v in results:
            venues_list.append((
             post,
             borough,
            neighborhood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']))

toronto_venues = pd.DataFrame(venues_list)
toronto_venues.columns = ['PostalCode', 
                         'Borough', 
                         'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

In [50]:
print(toronto_venues.shape)
toronto_venues.head()

(2194, 9)


,PostalCode,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,East Toronto,The Beaches,43.676357,-79.39442,Joso's,43.674835,-79.396171,Mediterranean Restaurant
1,M4E,East Toronto,The Beaches,43.676357,-79.39442,L'Unita Restaurant,43.674387,-79.396488,Italian Restaurant
2,M4E,East Toronto,The Beaches,43.676357,-79.39442,Ramsden Park,43.676068,-79.389705,Park
3,M4E,East Toronto,The Beaches,43.676357,-79.39442,Greenhouse Juice Co,43.679101,-79.390686,Juice Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.39442,Black Camel,43.677016,-79.389367,BBQ Joint


### Venues count by Postal code/Borough and Neighborhood grouping

In [51]:
toronto_venues.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,30,30,30,30,30,30
M4J,East York/East Toronto,The Danforth East,53,53,53,53,53,53
M4K,East Toronto,"The Danforth West, Riverdale",40,40,40,40,40,40
M4L,East Toronto,"India Bazaar, The Beaches West",100,100,100,100,100,100
M4M,East Toronto,Studio District,55,55,55,55,55,55
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,28,28,28,28,28,28
M4R,Central Toronto,North Toronto West,13,13,13,13,13,13
M4S,Central Toronto,Davisville,62,62,62,62,62,62


In [52]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 171 uniques categories.


In [53]:
toronto_venues['Venue Category'].unique()[:50]

array(['Mediterranean Restaurant', 'Italian Restaurant', 'Park',
       'Juice Bar', 'BBQ Joint', 'Vegetarian / Vegan Restaurant',
       'Thai Restaurant', 'Bar', 'Bank', 'Grocery Store', 'Coffee Shop',
       'French Restaurant', 'American Restaurant', 'Café',
       'Mexican Restaurant', 'Pharmacy', 'Pub',
       'Paper / Office Supplies Store', 'Tea Room', 'Pet Store',
       'Asian Restaurant', 'Breakfast Spot', 'Sporting Goods Shop',
       'Bagel Shop', 'Spa', 'Bakery', 'Sushi Restaurant', 'Liquor Store',
       'Burger Joint', 'Modern European Restaurant', 'German Restaurant',
       'Sandwich Place', 'Yoga Studio', 'Restaurant', 'Pizza Place',
       'Japanese Restaurant', 'Vietnamese Restaurant', 'Gym',
       'Cantonese Restaurant', 'Salad Place', 'Gym / Fitness Center',
       'Shopping Mall', 'Metro Station', 'Furniture / Home Store',
       'Wine Bar', 'Dessert Shop', 'Butcher', 'Gourmet Shop', 'Bistro',
       'Fish Market'], dtype=object)

### Analyse each area

In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Theater,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Plane,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [55]:
toronto_onehot.shape

(2194, 172)

In [56]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Theater,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Plane,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.016667,0.00,0.000000,0.016667,0.0,0.0,0.0,0.016667,0.016667,0.0,0.0,0.016667,0.000000,0.0,0.0,0.000000,0.016667,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.033333,0.0,0.0,0.016667,0.0,0.000000,0.0,0.00,0.00,0.083333,0.0,0.000000,0.00,0.016667,0.00,0.0,0.016667,0.0,0.0,0.0,0.016667,0.00,0.0,0.016667,0.016667,0.0,0.016667,0.0,0.00,0.000000,0.000000,0.016667,0.0,0.016667,0.0,0.016667,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.00,0.000000,0.016667,0.033333,0.033333,0.000000,0.0,0.033333,0.0,0.0,0.016667,0.016667,0.0,0.0,0.0,0.083333,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.016667,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.016667,0.016667,0.0,0.000000,0.0,0.050000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.033333,0.0,0.000000,0.00,0.033333,0.000000,0.000000,0.016667,0.016667,0.00,0.0,0.0,0.05,0.016667,0.0,0.0,0.016667,0.0,0.016667,0.016667,0.0,0.000000,0.016667,0.00,0.000000,0.016667,0.0,0.0,0.000000,0.0,0.016667,0.000000,0.0,0.000000,0.0,0.00,0.00,0.016667
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.030303,0.000000,0.0,0.0,0.000000,0.030303,0.0,0.0,0.030303,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.030303,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00,0.181818,0.0,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.030303,0.0,0.000000,0.0,0.00,0.030303,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.030303,0.0,0.00,0

In [57]:
toronto_grouped.shape

(39, 172)

In [58]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                  venue  freq
0    Italian Restaurant  0.08
1           Coffee Shop  0.08
2           Pizza Place  0.05
3                   Spa  0.05
4  Gym / Fitness Center  0.03


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0          Coffee Shop  0.18
1        Grocery Store  0.06
2                 Park  0.06
3                Track  0.03
4  Japanese Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4             Plane  0.06


----Central Bay Street----
                           venue  freq
0                    Coffee Shop  0.33
1                 Sandwich Place  0.10
2  Vegetarian / Vegan Restaurant  0.07
3                    Salad Place  0.03
4             Chinese Restaurant  0.03


----Christie----
                

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Theater,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Light Rail Station,Liquor Store,Lounge,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Plane,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.016667,0.00,0.000000,0.016667,0.0,0.0,0.0,0.016667,0.016667,0.0,0.0,0.016667,0.000000,0.0,0.0,0.000000,0.016667,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.033333,0.0,0.0,0.016667,0.0,0.000000,0.0,0.00,0.00,0.083333,0.0,0.000000,0.00,0.016667,0.00,0.0,0.016667,0.0,0.0,0.0,0.016667,0.00,0.0,0.016667,0.016667,0.0,0.016667,0.0,0.00,0.000000,0.000000,0.016667,0.0,0.016667,0.0,0.016667,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.00,0.000000,0.016667,0.033333,0.033333,0.000000,0.0,0.033333,0.0,0.0,0.016667,0.016667,0.0,0.0,0.0,0.083333,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.016667,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.016667,0.016667,0.0,0.000000,0.0,0.050000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.033333,0.0,0.000000,0.00,0.033333,0.000000,0.000000,0.016667,0.016667,0.00,0.0,0.0,0.05,0.016667,0.0,0.0,0.016667,0.0,0.016667,0.016667,0.0,0.000000,0.016667,0.00,0.000000,0.016667,0.0,0.0,0.000000,0.0,0.016667,0.000000,0.0,0.000000,0.0,0.00,0.00,0.016667
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.030303,0.000000,0.0,0.0,0.000000,0.030303,0.0,0.0,0.030303,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.030303,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00,0.181818,0.0,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.030303,0.0,0.000000,0.0,0.00,0.030303,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.030303,0.0,0.00,0

#### Now let's create the new dataframe and display the top 10 venues for each PostalCode.

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Italian Restaurant,Pizza Place,Spa,Restaurant,Gym,Gym / Fitness Center,Hotel,Café,Sandwich Place
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Grocery Store,Park,Gym,Restaurant,Ramen Restaurant,Pub,Pharmacy,Light Rail Station,Japanese Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique
3,Central Bay Street,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Thai Restaurant,Bar,Bank,Gaming Cafe,Sushi Restaurant,Salad Place
4,Christie,Clothing Store,Italian Restaurant,Restaurant,Café,Coffee Shop,French Restaurant,Sandwich Place,Boutique,Hotel,Spa


In [62]:
### Cluster Areas
#### Run k-means to cluster the Toronto areas into 5 clusters

In [63]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 4, 2, 0, 0, 0, 3, 3, 0])

In [64]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Italian Restaurant,Coffee Shop,Bank,French Restaurant,Café,Juice Bar,Mediterranean Restaurant,Mexican Restaurant,Paper / Office Supplies Store,Park
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,3,Coffee Shop,Thai Restaurant,Grocery Store,Sushi Restaurant,Italian Restaurant,Pizza Place,Gym,Spa,Bagel Shop,Bank
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Coffee Shop,Italian Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Spa,Mexican Restaurant,Pet Store,Pharmacy,Fish Market,Bistro
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Coffee Shop,Clothing Store,Restaurant,Café,French Restaurant,Sandwich Place,Italian Restaurant,Boutique,Spa,Burger Joint
4,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Coffee Shop,Café,Sandwich Place,Theater,Vegetarian / Vegan Restaurant,Burrito Place,Pub,Japanese Restaurant,French Restaurant,Road


### Visualizing the resulting clusters on map

In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining Clusters
#### Cluster 1

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,0,Coffee Shop,Italian Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Spa,Mexican Restaurant,Pet Store,Pharmacy,Fish Market,Bistro
3,East Toronto,0,Coffee Shop,Clothing Store,Restaurant,Café,French Restaurant,Sandwich Place,Italian Restaurant,Boutique,Spa,Burger Joint
11,Downtown Toronto,0,Coffee Shop,Italian Restaurant,French Restaurant,Vegetarian / Vegan Restaurant,Spa,Mexican Restaurant,Pet Store,Pharmacy,Fish Market,Bistro
12,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Restaurant,Italian Restaurant,Sandwich Place,Park,Boutique,French Restaurant,American Restaurant
13,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Sandwich Place,Restaurant,Boutique,Hotel,Italian Restaurant,Park,College Theater
14,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Art Gallery,Dessert Shop,Arts & Crafts Store,Chinese Restaurant,Pizza Place,Farmers Market,Breakfast Spot
16,Downtown Toronto,0,Coffee Shop,Sandwich Place,Clothing Store,Shoe Store,Art Gallery,Arts & Crafts Store,Pizza Place,Restaurant,Sushi Restaurant,Ramen Restaurant
17,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Pizza Place,Spa,Restaurant,Gym,Gym / Fitness Center,Hotel,Café,Sandwich Place
19,Downtown Toronto,0,Coffee Shop,Café,Art Gallery,Shoe Store,Restaurant,Pizza Place,Clothing Store,Burrito Place,Furniture / Home Store,Italian Restaurant
20,Downtown Toronto,0,Coffee Shop,Gym,Park,Gym / Fitness Center,French Restaurant,Japanese Restaurant,Baseball Stadium,New American Restaurant,Hotel,Mobile Phone Shop


#### Cluster 2

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,1,Pool,Pharmacy,Business Service,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dessert Shop


#### Cluster 3

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,2,Coffee Shop,Café,Sandwich Place,Theater,Vegetarian / Vegan Restaurant,Burrito Place,Pub,Japanese Restaurant,French Restaurant,Road
15,Downtown Toronto,2,Coffee Shop,Sandwich Place,Café,Chinese Restaurant,Sushi Restaurant,Burrito Place,Restaurant,Breakfast Spot,Farmers Market,Fast Food Restaurant
18,Downtown Toronto,2,Coffee Shop,Sandwich Place,Vegetarian / Vegan Restaurant,Arts & Crafts Store,Thai Restaurant,Bar,Bank,Gaming Cafe,Sushi Restaurant,Salad Place
26,Downtown Toronto,2,Coffee Shop,Café,Park,Burrito Place,Falafel Restaurant,Gym,Paper / Office Supplies Store,Gastropub,Mediterranean Restaurant,Theater
35,West Toronto,2,Coffee Shop,Pub,Café,Sandwich Place,Burrito Place,Gastropub,Bank,Mediterranean Restaurant,Theater,Bar
38,East Toronto Business,2,Coffee Shop,Café,Park,Burrito Place,Falafel Restaurant,Gym,Paper / Office Supplies Store,Gastropub,Mediterranean Restaurant,Theater


#### CLuster 4

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Italian Restaurant,Coffee Shop,Bank,French Restaurant,Café,Juice Bar,Mediterranean Restaurant,Mexican Restaurant,Paper / Office Supplies Store,Park
1,East York/East Toronto,3,Coffee Shop,Thai Restaurant,Grocery Store,Sushi Restaurant,Italian Restaurant,Pizza Place,Gym,Spa,Bagel Shop,Bank
6,Central Toronto,3,Coffee Shop,Italian Restaurant,Thai Restaurant,Diner,Food & Drink Shop,Café,Bakery,Clothing Store,Dog Run,Sporting Goods Shop
7,Central Toronto,3,Coffee Shop,Fast Food Restaurant,Thai Restaurant,Pizza Place,Food & Drink Shop,Breakfast Spot,Dog Run,Diner,Department Store,Café
8,Central Toronto,3,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Restaurant,Dessert Shop,Pizza Place,Middle Eastern Restaurant,Toy / Game Store,Salad Place
9,Central Toronto,3,Coffee Shop,Grocery Store,Café,Italian Restaurant,Bank,Bagel Shop,Sushi Restaurant,Pizza Place,Thai Restaurant,Gym
10,Central Toronto,3,Coffee Shop,Grocery Store,Italian Restaurant,Sushi Restaurant,Thai Restaurant,Bagel Shop,Pizza Place,Café,Bank,Gym
23,Central Toronto,3,Coffee Shop,Italian Restaurant,Café,Thai Restaurant,Sandwich Place,Food & Drink Shop,Bakery,Pizza Place,Hotel,Breakfast Spot
24,Central Toronto,3,Coffee Shop,Gastropub,Sushi Restaurant,Pub,Office,Japanese Restaurant,Mexican Restaurant,Falafel Restaurant,Sporting Goods Shop,Café
34,West Toronto,3,Coffee Shop,Grocery Store,Park,Gym,Restaurant,Ramen Restaurant,Pub,Pharmacy,Light Rail Station,Japanese Restaurant


#### Cluster 5

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique


#### Observations: Most of the neighborhoods fall into Cluster 1 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 2 is just a pool and pharmacy, Cluster 3 are coffee shops and burrito places, Cluster 4 restaurants and grocery shops and banks, and lastly Cluster 5 is Airport services.